# Tính toán tham số PDR và Latency cho 1 kênh    

Tạo môi trường python2 cho pyspark trên docker vì mặc định `pyspark-notebook` chọn python3 làm mặc định

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = '/opt/conda/envs/python2/bin/python'

## Import thư viện RDD của spark cho python và thư viện numpy

In [2]:
import pyspark
from pyspark import SparkContext
import numpy as np
sc = SparkContext('local[*]')

Load file **.csv** có định dạng là  **`Msg; TimeStamp; Id`**

In [3]:
# tf = sc.textFile('rlm_log_2_channel.csv')
tf = sc.textFile('rlm-2-channels.csv')

Sử dụng phép biến đổi map để biến đổi định dạng ban đầu thành 
<pre>
|   Key  |       Value       |
|--------|-------------------|
|   Msg  | Msg; TimeStamp; Id|
</pre>

Sau đó groupBy các Msg với nhau và những Msg giống nhau thì value của nó sẽ append vào 1 list

*Ví dụ*:
<pre>
 (u'Hello 7 from the client 6',
      [u'Hello 7 from the client 6;292614;6',
       u'Hello 7 from the client 6;294122;1']),
 (u'Hello 7 from the client 7', 
      [u'Hello 7 from the client 7;272781;7']),
 (u'Hello 10 from the client 3',
      [u'Hello 10 from the client 3;366856;3',
       u'Hello 10 from the client 3;367274;9',
       u'Hello 10 from the client 3;367589;1']),
 (u'Hello 4 from the client 9', 
      [u'Hello 4 from the client 9;200055;1']),
 (u'Hello 6 from the client 4', 
      [u'Hello 6 from the client 4;252735;4'])
</pre>

In [4]:
rdd = tf.map(lambda x: (x.split(";")[0], x)).groupByKey().mapValues(list)

Nhận thấy rằng khi `groupBy` theo `Msg`
- tổng số **key** có nhiều hơn 1 **value** (chắc chắn trong các value có id = 1) thì đó là số lượng packet nhận được của **`node 1`**  => recv
- tổng số **key** không có **value** đầu tiên có id = 1 thì đó là số lượng packet gửi đi của các node client => sent

### Vậy PDR = 100 * recv / sent

In [5]:
recv = rdd.filter(lambda x: len(x[1]) > 1).count()
sent = rdd.filter(lambda x: x[1][0].split(";")[2] != "1").count()
print 'PDR = ', recv / float(sent)

PDR =  0.88785046729


### Tính Latency theo từng hop

Nhận thấy rằng khi `groupBy` theo `Msg`

- số **key** có 2 value là 1 hop
- số **key** có 3 value là 2 hop

==> Dùng phép biến đổi `filter` để lọc ra số hop

In [6]:
hopping1 = rdd.filter(lambda x: len(x[1]) == 2)
hopping2 = rdd.filter(lambda x: len(x[1]) == 3)

### Latency =  Thời gian packet bắt đầu nhận - Thời gian packet bắt đầu phát

**Vì dữ liệu ban đầu đã được sắp xếp lần lượt theo thứ tự thời gian** nên độ trễ của
- `1 hop` lấy thời gian của **value** thứ 2 trừ đi thời gian của **value** thứ nhất
- `2 hop` lấy thời gian của **value** thứ 3 trừ đi thời gian của **value** thứ nhất (thời gian của **value** thứ 2 là thời gian nhận được của node trung gian để chuyển tiếp)

Sau đó dùng hàm **`collect()`** để export kết quả của phép biến đổi **`map`** sang dạng mảng và lưu vào biến `latency1` và `latency2`

In [7]:
latency1 = hopping1.map(lambda x: int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1])).collect()
latency2 = hopping2.map(lambda x: int(x[1][2].split(";")[1]) - int(x[1][0].split(";")[1])).collect()

### Sử dụng thư viện numpy để tính trung bình, độ lệch chuẩn, phương sai cho độ trễ của 1 hop và 2 hop

In [8]:
print 'mean latency 1: %f' % np.mean(latency1)
print 'mean latency 2: %f' % np.mean(latency2)
print 'Standard deviation latency 1: %f' % np.std(latency1)
print 'Standard deviation latency 2: %f' % np.std(latency2)
print 'Variance latency 1: %f' % np.var(latency1)
print 'Variance latency 2: %f' % np.var(latency2)

mean latency 1: 815.215686
mean latency 2: 1345.863636
Standard deviation latency 1: 1018.872740
Standard deviation latency 2: 1001.668099
Variance latency 1: 1038101.659362
Variance latency 2: 1003338.981405


## Sử dụng plotly để vẽ kết quả ra đồ thị trên Jupyter notebook

In [9]:
import plotly
from plotly.graph_objs import Scatter, Layout, Bar, Figure, Layout

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
# import numpy as np

init_notebook_mode(connected=True)

### Biểu đồ PDR và Latency

In [10]:
trace1 = Bar(
    x=['PDR'],
    y=[recv / float(sent)],
    name='Packet Delivery Rate'
)

data = [trace1]
layout = Layout(
    title = 'Biểu đồ PDR',
    barmode = 'group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='jupyter/PDR')


trace2 = Bar(
    x=['Latency'],
    y=[np.mean(latency1 + latency2)],
    name='Latency AVG'
)

data = [trace2]
layout = Layout(
    title = 'Biểu đồ Latency',
    barmode = 'group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='jupyter/Latency')

### Biểu đồ so sánh giữa 1 Hop và 2 Hop

In [11]:
trace1 = Bar(
    x=['1 Hop', '2 Hop',],
    y=[np.mean(latency1), np.mean(latency2)],
    name='Number of hop',
    error_y=dict(
        type='data',
        array=[np.std(latency1), np.std(latency2)],
        visible=True
    )
)

data = [trace1]
layout = Layout(
    title = 'Biểu đồ so sánh giữa 1 Hop và 2 Hop',
    barmode = 'group'
)
fig = Figure(data=data, layout=layout)
iplot(fig, filename='jupyter/Latency-per-hop')

### Biểu đồ độ trễ của từng packet theo từng hop

In [12]:
hop1 = hopping1.map(lambda x: (x[1][0].split(";")[2] + '-' + x[1][0].split(";")[1], (int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1]))))
print 'Hop1 is : ', hop1.take(5)
time2hop1 = hop1.sortByKey().map(lambda x: x[0]).collect()
latency2hop1 = hop1.sortByKey().map(lambda x: x[1]).collect()

hop2 = hopping2.map(lambda x: (x[1][0].split(";")[2] + '-' + x[1][0].split(";")[1], (int(x[1][1].split(";")[1]) - int(x[1][0].split(";")[1]))))
print 'Hop2 is : ', hop2.take(5)
time2hop2 = hop2.sortByKey().map(lambda x: x[0]).collect()
latency2hop2 = hop2.sortByKey().map(lambda x: x[1]).collect()

# draw to plotly
trace1 = Scatter(
    x = time2hop1,
    y = latency2hop1,
    mode = 'lines+markers',
    name = "1 Hop"
)
trace2 = Scatter(
    x = time2hop2,
    y = latency2hop2,
    mode = 'lines+markers',
    name = "2 Hop"
)
data = [trace1, trace2]
iplot(data, filename='jupyter/Latency-per-link')

Hop1 is :  [(u'6-284169', 74), (u'9-269338', 1346), (u'8-241834', 1103), (u'2-259881', 608), (u'9-287911', 430)]
Hop2 is :  [(u'7-293296', 301), (u'3-378298', 1168), (u'7-377703', 81), (u'3-255419', 447), (u'5-264906', 524)]
